In [1]:
#Prior to execute this notebook, please make sure you have already cloned the sample data
#!git clone https://github.com/xieyongliang/all-in-one-ai-sample-data.git -t ../../..

In [2]:
import os
import sagemaker
from sagemaker.mxnet import MXNet

In [3]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
!rm -rf sourcedir.tar.gz
!tar czvf sourcedir.tar.gz *

In [5]:
source_dir = 's3://{0}/{1}/source/'.format(bucket, 'gluoncv')
!aws s3 cp sourcedir.tar.gz $source_dir
source_dir = '{0}sourcedir.tar.gz'.format(source_dir)
!rm -f sourcedir.tar.gz

upload: ./sourcedir.tar.gz to s3://sagemaker-ap-east-1-034068151705/gluoncv/source/sourcedir.tar.gz


In [ ]:
train_dir = 's3://{0}/{1}/data/train'.format(bucket, 'gluoncv')
!cd ../../../all-in-one-ai-sample-data/gluoncv/
!bash ./build.sh
!aws s3 cp test $train_dir/test/ --recursive
!aws s3 cp train $train_dir/train/ --recursive

In [14]:
job_name = None
entry_point = 'finetune.py'
git_config = None
role = role
hyperparameters = {
    'classes': 101, 
    'batch_size': 8,
    'epochs': 20, 
    'learning_rate': 0.001,
    'momentum': 0.9,
    'wd': 0.0001,
    'num_workers': 8,
    'model_name': 'ResNet50_v2'}
framework_version = '1.8.0'
py_version = 'py37'
instance_type = 'ml.g4dn.16xlarge'
instance_count = 1
inputs = {
    'train': train_dir,
    'test': train_dir,
    'val': train_dir
}

In [15]:
estimator = MXNet(
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    debugger_hook_config = False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count
)

In [ ]:
estimator.fit(inputs, job_name = job_name)

In [17]:
training_job_name = estimator.latest_training_job.name

In [22]:
model_name = None
model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name)
entry_point = 'inference.py'
framework_version = '1.9.0'
py_version = 'py38'
model_environment = {
    'classes': '101'
}

In [23]:
from sagemaker.mxnet.model import MXNetModel

model = MXNetModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

In [24]:
endpoint_name = None
instance_type = 'ml.m5.xlarge'
instance_count = 1

In [ ]:
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count
)

In [27]:
from os import listdir
from os.path import isfile, join

sample_dir ='../../../all-in-one-ai-sample-data/gluoncv/images/chocolate_cake'
files = [f for f in listdir(sample_dir) if isfile(join(sample_dir, f))]

bytes  = open(join(sample_dir, files[0]), 'rb').read()

In [ ]:
from sagemaker.serializers import DataSerializer 
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = DataSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(bytes, initial_args={"ContentType":"image/png"})

In [ ]:
predictor.delete_endpoint()